In [10]:
import argparse
from path import Path

import torch
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import models
from tqdm import tqdm

import torchvision.transforms as transforms
from imageio import imread, imwrite
import numpy as np
from util import flow2rgb

import torch.nn as nn
from torch.nn.init import kaiming_normal_, constant_

from convlstm import ConvLSTM

from deepVO_wMemory import DvoAm_EncPlusTrack, DvoAm_Encoder

import torch.optim as optim

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

FlowNet_data = torch.load('flownets_EPE1.951.pth',map_location=torch.device('cpu'))
print("=> using pre-trained model '{}'".format(FlowNet_data["arch"]))
FlowNetModel = models.__dict__[FlowNet_data["arch"]](FlowNet_data).to(device)
#print(model)
#FlowNetModel.eval()
#print(FlowNetModel.state_dict()['conv1.0.weight'].data.size())
#print(FlowNetModel.state_dict()['conv1.0.weight'].data)
#print(FlowNetModel.named_parameters())

model2 = DvoAm_EncPlusTrack(batchNorm=False)


=> using pre-trained model 'flownets'


In [9]:
#Capturing the layers we want from FlowNet!
encodingLayers = ['conv1.0.weight', 'conv2.0.weight', \
                  'conv3.0.weight', 'conv3_1.0.weight', \
                  'conv4.0.weight', 'conv4_1.0.weight', \
                  'conv5.0.weight', 'conv5_1.0.weight', \
                  'conv6.0.weight',\
                  'conv1.0.bias', 'conv2.0.bias', \
                  'conv3.0.bias', 'conv3_1.0.bias', \
                  'conv4.0.bias', 'conv4_1.0.bias', \
                  'conv5.0.bias', 'conv5_1.0.bias', \
                  'conv6.0.bias' ]
subset_state_dict = {}
for name, param in FlowNetModel.named_parameters():
    if name in encodingLayers:  # Change 'desired_layer' to the relevant layer names
        subset_state_dict[name] = param
#print(subset_state_dict)


new_state_dict = model2.state_dict()
for name, param in subset_state_dict.items():
    #print(name)
    newName = 'encoding.'+name
    if newName in new_state_dict:
        new_state_dict[newName].copy_(param) #copy_() performs copy in place
print(new_state_dict.keys())
model2.load_state_dict(new_state_dict)

#model2.state_dict()['conv1.0.weight'] = FlowNetModel.state_dict()['conv1.0.weight']
#print(model2.state_dict()['encoding.conv1.0.weight'])
#print(model2.state_dict().keys())

odict_keys(['encoding.conv1.0.weight', 'encoding.conv1.0.bias', 'encoding.conv2.0.weight', 'encoding.conv2.0.bias', 'encoding.conv3.0.weight', 'encoding.conv3.0.bias', 'encoding.conv3_1.0.weight', 'encoding.conv3_1.0.bias', 'encoding.conv4.0.weight', 'encoding.conv4.0.bias', 'encoding.conv4_1.0.weight', 'encoding.conv4_1.0.bias', 'encoding.conv5.0.weight', 'encoding.conv5.0.bias', 'encoding.conv5_1.0.weight', 'encoding.conv5_1.0.bias', 'encoding.conv6.0.weight', 'encoding.conv6.0.bias', 'convLSTM.cell_list.0.conv.weight', 'convLSTM.cell_list.0.conv.bias', 'fc.weight', 'fc.bias'])


<All keys matched successfully>

In [11]:
def lossFunction(yPred, yGt):
    lossLocal = 1
    lossGlobal = 2
    loss = lossLocal +  lossGlobal
    return loss

In [13]:
# learning rate decays every 60k iterations
learningRateInitial = 0.0001
optimizer = optim.Adam(model2.parameters(), lr=0.00001, betas=(0.9, 0.99), weight_decay=0.0004)  # Adjust the LR as needed

numEpochs = 10
batchSize = 4
def train():
    for epoch in range(numEpochs):
        for i in range(0, len(X), batchSize):
            batchX = 3#....
            batchY = 3#....

            optimizer.zero_grad()
            outputs = model2(batchX)
            loss = lossFunction()
            loss.backwards()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{numEpochs}], Loss: {loss.item():.4f}")



In [ ]:
#prepare datasets:
train_images = []
test_images = []